<a href="https://colab.research.google.com/github/zrshen/ML_Colab/blob/main/ul_cw_dimensionality_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dimensionality Reduction**

# Setup 📋

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# Filter warnings
import warnings
warnings.filterwarnings('ignore')

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "dim_reduction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Exercises ✏️

**Load the MNIST dataset and split it into a training set and a test set (take the first 5,000 instances for training, and the next 2,500 for testing).**

In [ ]:
from sklearn.datasets import fetch_openml

# Load data and cast target into an numerical type
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

In [ ]:
# Manually separate X and y into train and test
X_train = mnist['data'][:5000]
y_train = mnist['target'][:5000]

X_test = mnist['data'][5000:7500]
y_test = mnist['target'][5000:7500]

**Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set.**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate the model
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
import time

# Time the model training
t0 = time.time()
rnd_clf.fit(X_train, y_train)
t1 = time.time()
print("Training took {:.2f}s".format(t1 - t0))

Training took 2.91s


In [ ]:
from sklearn.metrics import accuracy_score

# Check accuracy score
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9368

**Next, use PCA to reduce the dataset's dimensionality, with an explained variance ratio of 95%.**

Refresher: what is PCA? Principal Component Analysis is a technique to reduce the dimensionality of data. 

Conceptually, PCA extracts "new", more valuable features from existing data by mapping existing data a new space. The axes in this new space are along the lines of the most variance in the original space. Each of these new axes have an order of importance, and are orthogonal (uncorrelated) to each other. If you have 30-dimensional data, PCA will spit out 30 new axes. However, you can use fewer than 30, let's say half (15) to capture MORE than half of the variance in the data in a smaller (lower dimensional) format.

Mathematically, PCA works by performing eigenvalue decomposition on the X (data) matrix. Each principal component (axis) is described by the eigenvector corresponding the largest eigen value, second largest, third largest, and so on.

Strengths:
- PCA can simplify your data dramatically, improving training and prediction time.

Weaknesses:
- Can provide a different result on the same data than a technique such as LDA. Be careful about which technique you use, since they optimize for different things.
- Interpretation of results becomes impossible. Principle components are not features a person can conceptually map to real world interpretations in any practical way.
- Can dampen predictive signals in the data

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

**Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster?**

In [ ]:
rnd_clf2 = RandomForestClassifier(n_estimators=100, random_state=42)
t0 = time.time()
rnd_clf2.fit(X_train_reduced, y_train)
t1 = time.time()
print("Training took {:.2f}s".format(t1 - t0))

Training took 5.84s


🛑 Oh no! Training is actually more than twice slower now! How can that be? 🛑

Dimensionality reduction does not always lead to faster training time: it depends on the dataset, the model and the training algorithm.

If we try a softmax classifier instead of a random forest classifier, we may find that the training time is reduce when using PCA, but let's first check the accuracy of the random forest classifier.

**Next, evaluate the classifier on the test set: how does it compare to the previous classifier?**

In [ ]:
# Transform and predict on PCA data
X_test_reduced = pca.transform(X_test)
y_pred = rnd_clf2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.9036

It is common for performance to drop slightly when reducing dimensionality, because we do lose some useful signal in the process. However, the performance drop was not too bad in this case. Sometimes, it can be a lot worse even with 95% of the explained variance ratio like we just used. With higher training time and lower accuracy, PCA really did not help: it slowed down training and reduced performance ☹️

Let's see if it helps when using softmax regression:

In [ ]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)
t0 = time.time()
log_clf.fit(X_train, y_train)
t1 = time.time()

In [ ]:
print("Training took {:.2f}s".format(t1 - t0))

Training took 3.95s


In [ ]:
y_pred = log_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8712

As seen, softmax regression takes much longer to train on this dataset than the random forest classifier, and it performs worse on the test set. But that's not what we are interested in right now; we want to see how much PCA can help softmax regression. Let's train the softmax regression model using the reduced dataset:

In [ ]:
# Instantiate model
log_clf2 = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)

# Time training
t0 = time.time()
log_clf2.fit(X_train_reduced, y_train)
t1 = time.time()

In [ ]:
print("Training took {:.2f}s".format(t1 - t0))

Training took 1.19s


Nice! Reducing dimensionality led to over 2× speedup! 😊

Let's check the model's accuracy:

In [ ]:
y_pred = log_clf2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.8396

A very slight drop in performance, which might be a reasonable price to pay for a 2× speedup, depending on the application.

So there you have it: PCA can give you a formidable speedup...but not always!

## t-SNE and Isomap

Let's review these concepts quickly. 

t-SNE, or t-distributed Stochastic Neighbor Embedding is a technique primarily used to visalize high dimensional data in 2 or 3 dimensions. Conceptually, it tries to preserve clusters in high dimensions as it projects the data into lower dimensions. This is more than a simple strict projection onto an existing axis, which would not preserve the clusters very well as well as a more advanced technique like t-SNE.
Pros:
- Takes complex, high dimensional data and makes it easy to visualize in 2D or 3D

Cons:
- Final axes, like PCA, are not interpretable in a standard way, and are a means to an end (visualization) rather than a practical or useful tool in terms of explainability


Isomap reduction is a technique based on spectral theory. It is another dimensionality reduction technique that is usually efficient and very generalizable, which will we will compare to t-SNE and PCA.

Unlike PCA, t-SNE and Isomap Embeddings are non-linear dimensionality reduction techniques.

In [ ]:
# Instantiate and time PCA fit/transform time 
pca = PCA()
t0 = time.time()
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)
t1 = time.time()

print(f"PCA took {t1-t0} seconds to fit and transform the data")

# Instantiate and time model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)

t2 = time.time()
model.fit(X_train_reduced, y_train)
t3 = time.time()

y_pred = model.predict(X_test_reduced)
print(f"Model built on PCA data took {t3-t2} seconds with accuracy {accuracy_score(y_test, y_pred)}")

PCA took 1.3701550960540771 seconds to fit and transform the data
Model built on PCA data took 3.728423595428467 seconds with accuracy 0.8684


In [ ]:
from sklearn.manifold import TSNE

# Instantiate and time T-SNE fit/transform time 
tsne = TSNE(learning_rate='auto')
t0 = time.time()
X = mnist['data'][:7500]
X_reduced = tsne.fit_transform(X)
X_train_reduced, X_test_reduced = X_reduced[:5000], X_reduced[5000:7500]
t1 = time.time()

print(f"T-SNE took {t1-t0} seconds to fit and transform the data")

# Instantiate and time model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)

t2 = time.time()
model.fit(X_train_reduced, y_train)
t3 = time.time()

y_pred = model.predict(X_test_reduced)
print(f"Model built on T-SNE data took {t3-t2} seconds with accuracy {accuracy_score(y_test, y_pred)}")

T-SNE took 67.84546637535095 seconds to fit and transform the data
Model built on T-SNE data took 0.4148986339569092 seconds with accuracy 0.7964


In [ ]:
from sklearn.manifold import Isomap

# Instantiate and time Isomap fit/transform time 
isomap = Isomap(n_jobs=-1, n_neighbors=3)
t0 = time.time()
X_train_reduced = isomap.fit_transform(X_train)
X_test_reduced = isomap.transform(X_test)
t1 = time.time()

print(f"Isomap took {t1-t0} seconds to fit and transform the data")

# Instantiate and time model
model = LogisticRegression(multi_class="multinomial", solver="lbfgs", random_state=42)

t2 = time.time()
model.fit(X_train_reduced, y_train)
t3 = time.time()

y_pred = model.predict(X_test_reduced)
print(f"Model built on Isomap data took {t3-t2} seconds with accuracy {accuracy_score(y_test, y_pred)}")

Isomap took 19.256809949874878 seconds to fit and transform the data
Model built on Isomap data took 0.5019376277923584 seconds with accuracy 0.5384


What can we conclude here?

In terms of training time: T-SNE > Isomap > PCA

In terms of accuracy: PCA > T-SNE > Isomap

The accuracy/latency tradeoff is a common one in ML, and your tolerance on either side of that tradeoff depend on the size of data, business requirements, hardware, etc...

In this specific case, I would personally choose PCA because I value accuracy over latency unless there is a good reason to do otherwise.

Compared to baseline, all methods take longer initially when fitting the dimensionality reduction estimators, but save time later on when transforming new data. Overall, the model will usually have less accuracy since information is lost with dimensionality reduction. In this case, you should experiment with increasing the number of components in T-SNE and Isomap (currently we are using the default of 2). The accuracy will almost certainly be a lot higher.

Disclaimer: training and transform time will vary a lot with number of dimensions or components that you choose to keep with dimensionality reduction. The numbers in this example are subject to change depending on those factors, so please keep that in mind when choosing which kind of dimensionality reduction to try when doing this yourself.